In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename):    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    img_mean=np.mean(img)
    img_std=np.std(img)  
    dcm = pydicom.dcmread(filename)       

    age = int(dcm.PatientAge)
    if age < 10 or age > 80:
        print('Image {} is rejected since {} is not in the range 10 to 80'.format(filename,age))
        print('\n')
        return None
    return img, img_mean,img_std
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    img = (img - img_mean)/img_std
    proc_img = resize(img, img_size)    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, 'r') as json_file:
        model_file = json_file.read()
        model = keras.models.model_from_json(model_file)
        model.load_weights(weight_path)    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prediction=model.predict(img)
    if (prediction>thresh):
        print("Pneumonia")
    else:
        print("not Pneumonia")      
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

In [4]:
IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16

my_model = load_model(model_path,weight_path)
thresh = 0.66 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    if(check_dicom(i)!= None):
        img,img_mean,img_std = check_dicom(i)
    
    if check_dicom(i) is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Image test1.dcm is rejected since 81 is not in the range 10 to 80


Load file test1.dcm ...
Image test1.dcm is rejected since 81 is not in the range 10 to 80


Load file test2.dcm ...
Load file test2.dcm ...
Load file test2.dcm ...
not Pneumonia
[[0.5816595]]
Load file test3.dcm ...
Load file test3.dcm ...
Load file test3.dcm ...
not Pneumonia
[[0.5459813]]
Load file test4.dcm ...
Image test4.dcm is rejected since 81 is not in the range 10 to 80


Load file test4.dcm ...
Image test4.dcm is rejected since 81 is not in the range 10 to 80


Load file test5.dcm ...
Image test5.dcm is rejected since 81 is not in the range 10 to 80


Load file test5.dcm ...
Image test5.dcm is rejected since 81 is not in the range 10 to 80


Load file test6.dcm ...
Image test6.dcm is rejected since 81 is not in the range 10 to 80


Load file test6.dcm ...
Image test6.dcm is rejected since 81 is not in the range 10 to 80


